In [8]:
from bs4 import BeautifulSoup
import csv
import os
from os import mkdir
from os.path import exists, join
import urllib.request as request
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

datadir = "data"
if not exists(datadir):
    mkdir(datadir)

if not exists("/tmp"):
    mkdir("/tmp")

source = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cache = join("/tmp", "List_of_S%26P_500_companies.html")


def retrieve():
    request.urlretrieve(source, cache)


def extract():
    source_page = open(cache).read()
    soup = BeautifulSoup(source_page, "html.parser")
    table = soup.find("table", {"class": "wikitable sortable"})

    # Fail now if we haven't found the right table
    header = table.findAll("th")
    if header[0].text.rstrip() != "Symbol" or header[1].string != "Security":
        raise Exception("Can't parse Wikipedia's table!")

    # Retrieve the values in the table
    records = []
    symbols = []
    rows = table.findAll("tr")
    for row in rows:
        fields = row.findAll("td")
        if fields:
            symbol = fields[0].text.rstrip()
            # fix as now they have links to the companies on WP
            name = fields[1].text.replace(",", "")
            sector = fields[3].text.rstrip()
            records.append([symbol, name, sector])
            symbols.append(symbol + "\n")

    header = ["Symbol", "Name", "Sector"]
    writer = csv.writer(
        open("./data/constituents.csv", "w"), lineterminator="\n"
    )
    writer.writerow(header)
    # Sorting ensure easy tracking of modifications
    records.sort(key=lambda s: s[1].lower())
    writer.writerows(records)

    with open("./data/constituents_symbols.txt", "w") as f:
        # Sorting ensure easy tracking of modifications
        symbols.sort(key=lambda s: s[0].lower())
        f.writelines(symbols)


In [9]:
retrieve()
extract()

In [10]:
def file_to_list(file):
    rtn: object = []
    file_object: object = open(file, "r")
    rtn: object = file_object.read().splitlines()
    file_object.close()
    return list(filter(None, pd.unique(rtn).tolist())) # Remove Empty/Duplicates Values
    pass

# Example #    
data_from_file: object = file_to_list('/Users/nickd/Downloads/constituents_symbols.txt') 

In [ ]:
options = Options()
options.add_argument("--user-data-dir=/Users/nickd/Desktop/UserData")
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
for variable in data_from_file:
    link = "https://finance.yahoo.com/quote/"+variable+"/balance-sheet?p="+variable
    try:
        driver.get(link)
        time.sleep(2)
        button = driver.find_element(By.XPATH, "//button[contains(@class,'Fz(s) Fw(500) C($linkColor) P(0)')]")
        button.click()
        time.sleep(5)
    except NoSuchElementException:
        continue